### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
model_data.shape

(478328, 2831)

In [4]:
# check unqiue latest feature id
model_data['latest_feature_id'].nunique()

478328

In [5]:
model_data['unique_reference_number'].nunique()

478327

In [6]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [7]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]

In [8]:
model_df1.shape, oot_df1.shape

((39010, 2831), (12128, 2831))

In [9]:
features_list=[
'bounced_transactions_count',
'utilities_spending_pct',
'min_balance_last_3_months',
'num_debits_last_1_months',
'days_balance_lt_5000_M3',
'num_credit_gt_salary',
'cash_withdrawal_sum',
'days_balance_lt_10000_M3',
'ratio_balance_after_7_days_max_credit_M1',
'ratio_5th_day_balance_to_max_credit_M2',
'Max_Credit_Min_M1_M2_M3',
'sum_insurance_debit',
'balance_after_7_days_max_credit_M3',
'spend_vs_salary_ratio',
'month_year',
 '90_in_24mob']

In [10]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [11]:
oot_df = oot_df1[features_list]

In [12]:
model_df.shape, oot_df.shape

((39010, 16), (12128, 16))

In [13]:
model_df['month_year'].isnull().sum()

np.int64(0)

In [14]:
#Handling Null Values
for i in model_df.columns:
    model_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3025925/2889689820.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  model_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3025925/2889689820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[i].fillna(-9999,inplace=True)


In [15]:
#Handling Null Values
for i in oot_df.columns:
    oot_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3025925/1074864086.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  oot_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3025925/1074864086.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oot_df[i].fillna(-9999,inplace=True)


In [16]:
model_df['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
0.0    32389
1.0     6621
Name: count, dtype: int64

In [17]:
oot_df['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
0.0    10039
1.0     2089
Name: count, dtype: int64

In [18]:
#take out record only present in pg data(exclude null records)
model_df = model_df[(model_df !=-9999).all(axis=1)]

In [19]:
model_df.shape

(1808, 16)

In [20]:
oot_df = oot_df[(oot_df !=-9999).all(axis=1)]

In [21]:
oot_df.shape

(578, 16)

In [22]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

In [23]:
X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

In [24]:
X_oot.shape,y_oot.shape

((578, 14), (578,))

In [25]:
X_model.shape

(1808, 14)

### Split Model dataset into Train and Test

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [27]:
X_train.shape, X_test.shape

((1084, 14), (724, 14))

In [28]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(867, 14) (217, 14) (724, 14)


In [29]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [40]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
    'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 7,
    'alpha': 4.8,
    'gamma': 5.6,
    # 'scale_pos_weight': 3.8,
    'random_state': 24   
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 100,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.50000	eval-auc:0.50000
[10]	train-auc:0.58543	eval-auc:0.57859
[20]	train-auc:0.58543	eval-auc:0.57859
[30]	train-auc:0.59768	eval-auc:0.60171
[40]	train-auc:0.59733	eval-auc:0.60051
[50]	train-auc:0.61266	eval-auc:0.61234
[60]	train-auc:0.61606	eval-auc:0.61728
[70]	train-auc:0.61606	eval-auc:0.61728
[80]	train-auc:0.61606	eval-auc:0.61728
[90]	train-auc:0.61606	eval-auc:0.61728


In [41]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.62, Gini: 0.23, KS: 0.18
Eval AUC: 0.62, Gini: 0.23, KS: 0.21
Test AUC: 0.64, Gini: 0.29, KS: 0.25
OOT AUC: 0.64, Gini: 0.27, KS: 0.24


In [42]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.62, Gini: 0.2321, KS: 0.1797
Test AUC: 0.64, Gini: 0.2867, KS: 0.2531
OOT AUC: 0.64, Gini: 0.2718, KS: 0.2424


In [50]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                             Feature  Importance
2  MEAN(raw.amount WHERE device_os_category = Win...        16.0
1  SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...         6.0
0                                  distinct_merchant         5.0
5                                 max_one_time_spent         4.0
3  NUM_UNIQUE(pg_success_raw_data.month_end_start...         2.0
8                    STD(pg_success_raw_data.amount)         2.0
6                              amt_first_android_txn         2.0
4                      ENTROPY(raw.mode_cleaned)#720         1.0
7                    MAX(pg_success_raw_data.amount)         1.0


In [51]:
feature_importance_df

Feature  Importance
2  MEAN(raw.amount WHERE device_os_category = Win...        16.0
1  SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...         6.0
0                                  distinct_merchant         5.0
5                                 max_one_time_spent         4.0
3  NUM_UNIQUE(pg_success_raw_data.month_end_start...         2.0
8                    STD(pg_success_raw_data.amount)         2.0
6                              amt_first_android_txn         2.0
4                      ENTROPY(raw.mode_cleaned)#720         1.0
7                    MAX(pg_success_raw_data.amount)         1.0

In [52]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.227011         0.192166        0.198337        0.188385    696   
7          0.172414         0.186924        0.188368        0.184905    696   
6          0.152958         0.182705        0.184843        0.181301    693   
5          0.155523         0.180417        0.181295        0.179791    688   
4          0.170904         0.179317        0.179766        0.179005    708   
3          0.141264         0.178722        0.178989        0.178550    538   
2          0.107492         0.178439        0.178541        0.178089    614   
1          0.118353         0.177784        0.178080        0.177667   2332   

        label_good  label_bad  
Decile                         
8              538        158  
7              576        120  
6              587        106  
5              581        107  
4              587        121  
3              462         76  
2              548         66  
1             2056        276

In [53]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.200698         0.191611        0.198337        0.187888    573   
7          0.157168         0.186474        0.187836        0.183821    579   
6          0.146010         0.182128        0.183801        0.180776    589   
5          0.130125         0.180240        0.180767        0.179601    561   
4          0.143101         0.179265        0.179596        0.178964    587   
3          0.148325         0.178725        0.178956        0.178550    418   
2          0.130682         0.178439        0.178541        0.178089    528   
1          0.124873         0.177791        0.178080        0.177667   1970   

        label_good  label_bad  
Decile                         
8              458        115  
7              488         91  
6              503         86  
5              488         73  
4              503         84  
3              356         62  
2              459         69  
1             1724        246

In [67]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.292135         0.201304        0.201542        0.200836     89   
7          0.279412         0.200592        0.200809        0.200054     68   
6          0.215385         0.199594        0.200034        0.199305     65   
5          0.140625         0.199064        0.199304        0.198584     64   
4          0.121951         0.197774        0.198554        0.197114     41   
3          0.106667         0.194493        0.194823        0.194136     75   
2          0.086957         0.193668        0.194085        0.193400     23   
1          0.117647         0.192840        0.193354        0.192670    153   

        label_good  label_bad  
Decile                         
8               63         26  
7               49         19  
6               51         14  
5               55          9  
4               36          5  
3               67          8  
2               21          2  
1              135         18

In [43]:
xgb_model.save_model('LeoPayu_BA_only_90_in_24mob.json')
print('Model Saved')

Model Saved
